In [174]:
from data import Data
from dimension_reduction import PCADimensionReduction

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics

In [175]:
# You can test the other datasets by changing these file names
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
train_indices_filepath = "data/SC_integration/train_indices.npy"
test_indices_filepath = "data/SC_integration/test_indices.npy"
SEED = 42
FOLD_NUMBER = 3
BATCH_SIZE = 8

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4
PCA_VARIABLES_AMOUNT = 60

# There are 4 data variants to check: regular, scaled, cut by max, pca reduced + cut by max
data_object = Data(data_filepath, true_results_filepath)
train_data, test_data, train_true_results, test_true_results =  data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)
scaled_train_data, scaled_test_data = data_object.get_scaled_train_test_data()

In [176]:
cut_by_max_train_data, cut_by_max_test_data = data_object.get_cut_by_max_train_test_data(CUT_BY_MAX_THRESHOLD)

pca_object = PCADimensionReduction(cut_by_max_train_data, scaled_train_data, train_true_results, SEED)
pca_variables = pca_object.get_most_important_variables_from_pc1(PCA_VARIABLES_AMOUNT)
pca_reduced_train_data = cut_by_max_train_data[pca_variables.index]
pca_reduced_test_data = cut_by_max_test_data[pca_variables.index]

In [177]:
def create_neural_network(input_shape):
    model = Sequential()
    model.add(Dense(1000, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.BinaryAccuracy(), metrics.AUC(), metrics.Precision(), metrics.Recall()])
    return model

In [178]:
def display_neural_network_metrics(model, batch_size_value, data, true_results, set_name):
    results = model.evaluate(data, true_results, batch_size=batch_size_value)

    print(f"{set_name} loss: {results[0]}")
    print(f"{set_name} binary accuracy: {results[1]}")
    print(f"{set_name} auc: {results[2]}")
    print(f"{set_name} precision: {results[3]}")
    print(f"{set_name} recall: {results[4]}")

## Neural Network

### Regular data

In [179]:
neural_network = create_neural_network(train_data.shape[1])
neural_network.fit(train_data, train_true_results, batch_size=BATCH_SIZE, epochs=100)

Epoch 1/100
385/385 [==============================] - 1s 2ms/step - loss: 0.1135 - binary_accuracy: 0.9873 - auc_41: 0.5026 - precision_41: 0.0000e+00 - recall_41: 0.0000e+00
Epoch 2/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0230 - binary_accuracy: 0.9941 - auc_41: 0.9438 - precision_41: 0.9048 - recall_41: 0.5429
Epoch 3/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0135 - binary_accuracy: 0.9980 - auc_41: 0.9556 - precision_41: 0.9677 - recall_41: 0.8571
Epoch 4/100
385/385 [==============================] - 1s 1ms/step - loss: 7.8076e-05 - binary_accuracy: 1.0000 - auc_41: 1.0000 - precision_41: 1.0000 - recall_41: 1.0000
Epoch 5/100
385/385 [==============================] - 1s 2ms/step - loss: 1.7921e-06 - binary_accuracy: 1.0000 - auc_41: 1.0000 - precision_41: 1.0000 - recall_41: 1.0000
Epoch 6/100
385/385 [==============================] - 1s 1ms/step - loss: 1.1887e-06 - binary_accuracy: 1.0000 - auc_41: 1.0000 - precision_41:

In [180]:
display_neural_network_metrics(neural_network, BATCH_SIZE, train_data, train_true_results, "Train")

385/385 [==============================] - 1s 1ms/step - loss: 7.2823e-11 - binary_accuracy: 1.0000 - auc_41: 1.0000 - precision_41: 1.0000 - recall_41: 1.0000
Train loss: 7.282344172132582e-11
Train binary accuracy: 1.0
Train auc: 1.0
Train precision: 1.0
Train recall: 1.0


In [181]:
display_neural_network_metrics(neural_network, BATCH_SIZE, test_data, test_true_results, "Test")

165/165 [==============================] - 0s 1ms/step - loss: 0.3357 - binary_accuracy: 0.9886 - auc_41: 0.5000 - precision_41: 0.0000e+00 - recall_41: 0.0000e+00
Test loss: 0.3357192277908325
Test binary accuracy: 0.9886191487312317
Test auc: 0.5
Test precision: 0.0
Test recall: 0.0


### Scaled data

In [182]:
neural_network = create_neural_network(scaled_train_data.shape[1])
neural_network.fit(scaled_train_data, train_true_results, batch_size=BATCH_SIZE, epochs=100)

Epoch 1/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0933 - binary_accuracy: 0.9902 - auc_42: 0.8447 - precision_42: 0.5676 - recall_42: 0.6000
Epoch 2/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0064 - binary_accuracy: 0.9993 - auc_42: 0.9857 - precision_42: 0.9714 - recall_42: 0.9714
Epoch 3/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0578 - binary_accuracy: 0.9967 - auc_42: 0.9703 - precision_42: 0.8378 - recall_42: 0.8857
Epoch 4/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0508 - binary_accuracy: 0.9980 - auc_42: 0.9709 - precision_42: 0.8919 - recall_42: 0.9429
Epoch 5/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0648 - binary_accuracy: 0.9964 - auc_42: 0.9703 - precision_42: 0.8158 - recall_42: 0.8857
Epoch 6/100
385/385 [==============================] - 1s 2ms/step - loss: 0.0011 - binary_accuracy: 0.9997 - auc_42: 1.0000 - precision_42: 0.9722 - recall_42:

In [183]:
display_neural_network_metrics(neural_network, BATCH_SIZE, scaled_train_data, train_true_results, "Test")

385/385 [==============================] - 1s 1ms/step - loss: 1.6008e-17 - binary_accuracy: 1.0000 - auc_42: 1.0000 - precision_42: 1.0000 - recall_42: 1.0000
Test loss: 1.6007710056328274e-17
Test binary accuracy: 1.0
Test auc: 1.0
Test precision: 1.0
Test recall: 1.0


In [184]:
display_neural_network_metrics(neural_network, BATCH_SIZE, scaled_test_data, test_true_results, "Test")

165/165 [==============================] - 0s 1ms/step - loss: 9.5691 - binary_accuracy: 0.9901 - auc_42: 0.7312 - precision_42: 0.5833 - recall_42: 0.4667
Test loss: 9.569055557250977
Test binary accuracy: 0.9901365637779236
Test auc: 0.7312100529670715
Test precision: 0.5833333134651184
Test recall: 0.46666666865348816


### Cut by max

In [185]:
neural_network = create_neural_network(cut_by_max_train_data.shape[1])
neural_network.fit(cut_by_max_train_data, train_true_results, batch_size=BATCH_SIZE, epochs=100)

Epoch 1/100
385/385 [==============================] - 1s 1ms/step - loss: 0.1021 - binary_accuracy: 0.9860 - auc_43: 0.4985 - precision_43: 0.0000e+00 - recall_43: 0.0000e+00
Epoch 2/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0738 - binary_accuracy: 0.9886 - auc_43: 0.5882 - precision_43: 0.0000e+00 - recall_43: 0.0000e+00
Epoch 3/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0610 - binary_accuracy: 0.9886 - auc_43: 0.6817 - precision_43: 0.0000e+00 - recall_43: 0.0000e+00
Epoch 4/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0538 - binary_accuracy: 0.9886 - auc_43: 0.8032 - precision_43: 0.0000e+00 - recall_43: 0.0000e+00
Epoch 5/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0473 - binary_accuracy: 0.9889 - auc_43: 0.8510 - precision_43: 1.0000 - recall_43: 0.0286
Epoch 6/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0374 - binary_accuracy: 0.9883 - auc_43: 0.9045 - p

In [186]:
display_neural_network_metrics(neural_network, BATCH_SIZE, cut_by_max_train_data, train_true_results, "Train")

385/385 [==============================] - 1s 1ms/step - loss: 3.2784e-10 - binary_accuracy: 1.0000 - auc_43: 1.0000 - precision_43: 1.0000 - recall_43: 1.0000
Train loss: 3.278401439210654e-10
Train binary accuracy: 1.0
Train auc: 1.0
Train precision: 1.0
Train recall: 1.0


In [187]:
display_neural_network_metrics(neural_network, BATCH_SIZE, cut_by_max_test_data, test_true_results, "Test")

165/165 [==============================] - 0s 1ms/step - loss: 0.1009 - binary_accuracy: 0.9947 - auc_43: 0.8658 - precision_43: 0.8333 - recall_43: 0.6667
Test loss: 0.10093561559915543
Test binary accuracy: 0.9946889281272888
Test auc: 0.8657969236373901
Test precision: 0.8333333134651184
Test recall: 0.6666666865348816


### PCA + cut by max

In [188]:
neural_network = create_neural_network(pca_reduced_train_data.shape[1])
neural_network.fit(pca_reduced_train_data, train_true_results, batch_size=BATCH_SIZE, epochs=100)

Epoch 1/100
385/385 [==============================] - 1s 1ms/step - loss: 0.1050 - binary_accuracy: 0.9886 - auc_44: 0.5669 - precision_44: 0.0000e+00 - recall_44: 0.0000e+00
Epoch 2/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0727 - binary_accuracy: 0.9886 - auc_44: 0.5842 - precision_44: 0.0000e+00 - recall_44: 0.0000e+00
Epoch 3/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0584 - binary_accuracy: 0.9886 - auc_44: 0.7212 - precision_44: 0.0000e+00 - recall_44: 0.0000e+00
Epoch 4/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0592 - binary_accuracy: 0.9886 - auc_44: 0.7258 - precision_44: 0.0000e+00 - recall_44: 0.0000e+00
Epoch 5/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0500 - binary_accuracy: 0.9886 - auc_44: 0.8205 - precision_44: 0.0000e+00 - recall_44: 0.0000e+00
Epoch 6/100
385/385 [==============================] - 1s 1ms/step - loss: 0.0471 - binary_accuracy: 0.9886 - auc_44: 0.

In [189]:
display_neural_network_metrics(neural_network, BATCH_SIZE, pca_reduced_train_data, train_true_results, "Train")

385/385 [==============================] - 1s 1ms/step - loss: 2.8204e-09 - binary_accuracy: 1.0000 - auc_44: 1.0000 - precision_44: 1.0000 - recall_44: 1.0000
Train loss: 2.820365052613738e-09
Train binary accuracy: 1.0
Train auc: 1.0
Train precision: 1.0
Train recall: 1.0


In [190]:
display_neural_network_metrics(neural_network, BATCH_SIZE, pca_reduced_test_data, test_true_results, "Test")

165/165 [==============================] - 0s 1ms/step - loss: 0.0935 - binary_accuracy: 0.9939 - auc_44: 0.7991 - precision_44: 0.8889 - recall_44: 0.5333
Test loss: 0.09350316226482391
Test binary accuracy: 0.9939302206039429
Test auc: 0.7991302013397217
Test precision: 0.8888888955116272
Test recall: 0.5333333611488342
